In [ ]:
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

# Run data_processing and batching_data
%run ~/violin-renderer/src/data/batching_data.ipynb
%run ~/violin-renderer/src/models/mlp/data_processing.ipynb

# Initialize GPU to move model/tensors onto
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
# Trains the model inputted into the function.

# @param model: The model object to be trained
# @param optimizer: The optimizing equation to use to train the model
# @param data_loader: values for source_input_note and ground_truth
# @param loss_module: Equation for calculating the difference between generated and actual output
def train_model_loop(model, optimizer, data_loader, loss_module):
    # Set model to train mode
    model.train()

    # Training loop   
    for batch, (source_input_note, ground_truth) in enumerate(data_loader):
        ## Step 1: Move input data to device (only strictly necessary if we use GPU)
        source_input_note = source_input_note.to(device)
        ground_truth = ground_truth.to(device)

        ## Step 2: Run the model on the input data
        predictions = model(source_input_note)

        ## Step 3: Calculate the loss
        loss = loss_module(predictions, ground_truth)

        ## Step 4: Perform backpropagation
        # Before calculating the gradients, we need to ensure that they are all zero.
        # The gradients would not be overwritten, but actually added to the existing ones.
        optimizer.zero_grad()
        loss.backward()

        ## Step 5: Update the parameters
        optimizer.step()

        ## Step 6: For every 50th batch, print out the current loss as well # of samples trained
        if batch % 50 == 0:
            loss = loss.item()
            samples_trained = batch * 100 + len(source_input_note)
            print(f"loss: {loss:>7f} [{samples_trained}/{len(data_loader.dataset)}]")

In [ ]:
# trains the model using the dataloader

# @param model: The model object to be trained
# @param optimizer: The optimizing equation to use to train the model
# @param data_loader: values for source_input_note and ground_truth
# @param loss_module: Equation for calculating the difference between generated and actual output
# @param name: name of model want to save as
def train_model(model, optimizer, training_loader, loss, name):
    epochs = 50
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_model_loop(model, optimizer, training_loader, loss)
    torch.save(model.state_dict(), HOME_PATH + '/src/models/mlp/states/' + name + '.pt')

In [ ]:
# calculates error between generated output and testing truth

# @param mlp_model: mlp model to use
# @return: an average MSE for all songs in the testing dataset
def MSE_error(mlp_model):
    testing_results = generate_all_testing_data(mlp_model)
    _ , testing_ground_truths = processed_testing_datasets()
    
    MSE_results = []
    for generative_timings, ground_truth in zip(testing_results.values(), testing_ground_truths.values()):
        predict_values = torch.Tensor(generative_timings)
        ground_truth_values = torch.Tensor(ground_truth)
        MSE_value = loss(predict_values, ground_truth_values) / len(predict_values)
        MSE_results.append(MSE_value)

    return MSE_results

In [ ]:
# this creates a dictionary of outputs for all pieces in the testing dataset

# @param mlp_model: mlp model to use
# @returns a dictionary mapping with key of file path and value of start and end
def generate_all_testing_data(mlp_model):
    mlp_model.to(device)
    testing_source_inputs, _ = processed_testing_datasets()

    # generating an output for each piece in the testing input dataset
    testing_results = {}
    for song_path, song_notes in testing_source_inputs.items():
        test_input = torch.Tensor(song_notes)
        test_input = test_input.to(device)

        # generate start and end timings
        predictions = mlp_model(test_input)

        # append to map
        predictions = predictions.tolist()
        testing_results[song_path] = predictions

    return testing_results

In [ ]:
# Get processed training data
training_source_inputs, training_ground_truths = processed_training_datasets()

# Initializing MLPMusicDataset objects
training_MLPMusicDataset = MLPMusicDataset(source_input_data=torch.Tensor(training_source_inputs), ground_truth_data=torch.Tensor(training_ground_truths))

# Load data to create batches
training_loader = DataLoader(training_MLPMusicDataset, batch_size=100, shuffle=True)

# initialize the MLP model
mlp_model = MLP(3, 4, 2)

# transfer model to GPU
mlp_model.to(device)

# Define our loss function (mean squared error) to be used in the grad descent step
loss = nn.MSELoss()

# Performs the gradient descent steps
optimizer = torch.optim.SGD(mlp_model.parameters(), lr=1e-6)

In [ ]:
# Train model
# train_model(mlp_model, optimizer, training_loader, loss, name="mlp_model_256_beat")